<a href="https://colab.research.google.com/github/faisalaldwaish/Sentiment-Analysis-of-Restaurant-Reviews/blob/main/gradio_review_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 966.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing i

In [ ]:
pip install fuzzywuzzy[speedup]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 60.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import gradio as gr
import seaborn as sns
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz

# Updated list of reviews and their associated ratings
reviews_data = [
    {"restaurant": "Beyond Flavours", "reviewer": "Rusha Chakraborty", "review": "The ambience was good, food was quite good.", "rating": 5, "metadata": "1 Review, 2 Followers", "time": "5/25/2019 15:54", "pictures": 0, "additional_info": 2447.0},
    {"restaurant": "Beyond Flavours", "reviewer": "Anusha Tirumalaneedi", "review": "Ambience is too good for a pleasant evening. Service is excellent.", "rating": 5, "metadata": "3 Reviews, 2 Followers", "time": "5/25/2019 14:20", "pictures": 0, "additional_info": None},
    {"restaurant": "Beyond Flavours", "reviewer": "Ashok Shekhawat", "review": "A must try.. great food great ambience. Thnx for the service.", "rating": 5, "metadata": "2 Reviews, 3 Followers", "time": "5/24/2019 22:54", "pictures": 0, "additional_info": None},
    {"restaurant": "Beyond Flavours", "reviewer": "Swapnil Sarkar", "review": "Soumen das and Arun was a great guy. Only because of them I am giving 5 stars.", "rating": 5, "metadata": "1 Review, 1 Follower", "time": "5/24/2019 22:11", "pictures": 0, "additional_info": None},
    {"restaurant": "Beyond Flavours", "reviewer": "Dileep", "review": "Food is good. We ordered Kodi drumsticks and baby corn.", "rating": 5, "metadata": "3 Reviews, 2 Followers", "time": "5/24/2019 21:37", "pictures": 0, "additional_info": None},
    {"restaurant": "Beyond Flavours", "reviewer": "Nagabhavani K", "review": "Ambiance is good, service is good, food is average.", "rating": 5, "metadata": "1 Review", "time": "5/24/2019 15:22", "pictures": 0, "additional_info": None},
    {"restaurant": "Beyond Flavours", "reviewer": "Jamuna Bhuwalka", "review": "It's a very nice place, ambience is different, and food is good.", "rating": 5, "metadata": "1 Review", "time": "5/24/2019 01:02", "pictures": 0, "additional_info": None},
    {"restaurant": "Beyond Flavours", "reviewer": "Sandhya S", "review": "Well after reading so many reviews finally visited. Food was good but service could be better.", "rating": 4, "metadata": "1 Review", "time": "5/23/2019 15:01", "pictures": 0, "additional_info": None},
    {"restaurant": "Beyond Flavours", "reviewer": "Akash Thorat", "review": "Excellent food, especially if you like spicy food. Service was quick.", "rating": 5, "metadata": "1 Review, 1 Follower", "time": "5/22/2019 23:12", "pictures": 0, "additional_info": None}
]

# Create a DataFrame from the reviews_data
reviews_df = pd.DataFrame(reviews_data)

# Function to classify a user's review based on dataset matching
def classify_review(user_review):
    try:
        if not user_review.strip():
            return "Please enter a valid review."

        best_match = None
        best_score = 0

        # Iterate through the reviews_data for matching
        for _, row in reviews_df.iterrows():
            if pd.isna(row['review']):
                continue

            # Use fuzzy matching to find the most similar review in the dataset
            score = fuzz.token_sort_ratio(user_review.lower(), str(row['review']).lower())
            if score > best_score:
                best_score = score
                best_match = row

        if best_score > 80:  # If the fuzzy match score is greater than 80
            rating = best_match['rating']
            # Classify based on the rating
            rating_based_classification = f"Positive review based on rating: {rating}" if int(rating) >= 4 else f"Negative review based on rating: {rating}"

            return f"{rating_based_classification}\nMatching Score: {best_score}%\nBest Match:\nRestaurant: {best_match['restaurant']}\nReviewer: {best_match['reviewer']}\nReview: {best_match['review']}\nRating: {best_match['rating']}"
        else:
            return "Review not found in the dataset."

    except Exception as e:
        return f"An error occurred: {str(e)}"

# Function to plot the distribution of ratings from the dataset
def plot_rating_distribution():
    plt.figure(figsize=(8, 6))
    sns.countplot(x='rating', data=reviews_df, order=[1, 2, 3, 4, 5])
    plt.title('Distribution of Ratings')
    plt.xlabel('Rating')
    plt.ylabel('Count')
    plt.tight_layout()
    return plt.gcf()

# Function to allow users to preview the dataset (first 10 rows)
def preview_dataset():
    return reviews_df.head(15)  # Return the first 10 rows of the dataset

# Create the Gradio interface for classifying reviews
review_interface = gr.Interface(
    fn=classify_review,
    inputs=gr.Textbox(lines=2, placeholder="Enter your review here", label="Reviews"),
    outputs="text",
    title="Review Classifier Based on Rating",
    description="Enter a restaurant review. The system will classify it based on the dataset rating."
)

# Create the Gradio interface for plotting the rating distribution
plot_interface = gr.Interface(
    fn=plot_rating_distribution,
    inputs=[],
    outputs="plot",
    title="Rating Distribution",
    description="Shows the distribution of ratings in the dataset."
)

# Create the Gradio interface for previewing the dataset
preview_interface = gr.Interface(
    fn=preview_dataset,
    inputs=[],
    outputs="dataframe",
    title="Preview Restaurant Reviews Dataset",
    description="Displays the first 10 rows of the dataset for preview."
)

# Combine all interfaces (Review Classifier, Rating Distribution, Dataset Preview) into tabs
tabbed_interface = gr.TabbedInterface(
    [review_interface, plot_interface, preview_interface],
    ["Review Classifier", "Rating Distribution", "Dataset Preview"]
)

# Launch the Gradio interface
tabbed_interface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://18c78bfc46de1acbf2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
